### Import Required Package

In [5]:
import pandas as pd
import talib
import numpy as np
import json

### Import Tidal

In [6]:
import tidal as td

### Initialize Plumber

In [7]:
from pathlib import Path
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

PLUMBER_HOST = "https://dev-api.ddt-dst.cc/api/plumber/"
with open(f'{str(Path.home())}/.config/gcloud/application_default_credentials.json') as plumber_token:
    token = json.load(plumber_token)

### Load market data and compute technical indicators

In [8]:
import os
import pandas as pd

os.chdir('/Users/jeff/Desktop/individualized-indicator')
# 定義你想要讀取檔案的目錄
directory = './emb'

start_date = "2022-06-01"
end_date = "2024-09-30"

# 初始化一個空的字典來存儲所有讀取的 DataFrame
dataframes = {}

# 遍歷目錄中的所有檔案
for filename in os.listdir(directory):
    if filename.endswith('.pkl'):
        # 組合完整的檔案路徑
        file_path = os.path.join(directory, filename)
        # 讀取 .pkl 檔案
        df = pd.read_pickle(file_path)
        
        # 使用檔案名稱（不包括副檔名）作為字典的鍵
        variable_name = filename.split('_')[2]
        
        # 將 DataFrame 儲存到字典中
        dataframes[variable_name] = df

keys_list = list(dataframes.keys())
# 將字符串轉換為 datetime 格式，然後進行排序
sorted_dates = sorted(pd.to_datetime(keys_list))
keys_list = [date.strftime('%Y-%m-%d') for date in sorted_dates]

total_df = pd.DataFrame()

for j in range(0,len(keys_list)):
    df_name = keys_list[j]
    selected_df = dataframes[df_name]    
    selected_df = selected_df.reset_index()
    stock_list = pd.DataFrame(selected_df.iloc[:,0].unique())
    total_df = pd.concat([total_df, stock_list], ignore_index=True)
    
unique_values_list = list(total_df.iloc[:,0].unique())
sorted_list = sorted(unique_values_list)

total_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": ",".join([stock for stock in sorted_list]),
    },
)

total_data.index = total_data.index.set_levels(
    pd.to_datetime(total_data.index.levels[1]),level=1)

total_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

total_data['new'] = -1
total_data.reset_index(inplace=True)

def filter_by_year_month_9(filter_data, select_year, select_mon):
    # 計算當前篩選年份和下一個年份
    next_year = select_year + 1
    
    # 構建篩選條件：當前年12月、次年1月、次年2月
    filter_data = filter_data[
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 1)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 2)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 3))
    ]
    
    return filter_data

def filter_by_year_month_12(filter_data, select_year, select_mon):
    # 計算當前篩選年份和下一個年份
    next_year = select_year + 1
    
    # 構建篩選條件：當前年12月、次年1月、次年2月
    filter_data = filter_data[
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 4)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 5)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 6))
    ]
    
    return filter_data

concat_df = pd.DataFrame()

# for i in range(0,len(keys_list)-1):
for i in range(0,len(keys_list)):
    df_name = keys_list[i]
    selected_df = dataframes[df_name]
    selected_df.reset_index(inplace=True)
    stock_df = selected_df.iloc[:,0]
    stock_df_unique = list(stock_df.unique())
    result = list(set(sorted_list) - set(stock_df_unique))
    filtered_df = total_data[total_data['instrument'].isin(result)]
    filtered_df = filtered_df.rename(columns={'volume': 'vol'})
    selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
    
    select_year = int(df_name.split('-')[0])
    select_mon = int(df_name.split('-')[1])
    select_mons = [(select_mon+4)%12, (select_mon+5)%12, (select_mon+6)%12]
    select_mons = [12 if mon == 0 else mon for mon in select_mons]
    
    selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
    # 過濾出所有6月的交易資料
    filter_data = selected_df[selected_df['datetime'].dt.month.isin(select_mons)]

    # 使用時，根據 select_year 和 select_mon 動態篩選數據
    
    if select_mon == 9:
        filter_data = filter_by_year_month_9(filter_data, select_year, select_mon)
    
    elif select_mon == 12:
        filter_data = filter_by_year_month_12(filter_data, select_year, select_mon) 

    else:
        filter_data = filter_data[filter_data['datetime'].dt.year == select_year]
    
    concat_df = pd.concat([concat_df, filter_data], ignore_index=True)
    
# i = len(keys_list)-1
# df_name = keys_list[i]
# selected_df = dataframes[df_name]
# selected_df.reset_index(inplace=True)
# stock_df = selected_df.iloc[:,0]
# stock_df_unique = list(stock_df.unique())
# result = list(set(sorted_list) - set(stock_df_unique))
# filtered_df = total_data[total_data['instrument'].isin(result)]
# filtered_df = filtered_df.rename(columns={'volume': 'vol'})
# selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
# concat_df = pd.concat([concat_df, selected_df], ignore_index=True)

concat_df.sort_values(by=['instrument', 'datetime'], inplace=True)
concat_df.reset_index(drop=True, inplace=True)
row = np.where(concat_df.iloc[:,0] == '6251')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
row = np.where(concat_df.iloc[:,0] == '8406')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
row = np.where(concat_df.iloc[:,0] == '6548')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
quote_data = concat_df.set_index(['instrument', 'datetime'])
quote_data.columns.values[4] = 'volume'

/tmp/ipykernel_37825/861545154.py:83: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
/tmp/ipykernel_37825/861545154.py:83: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
/tmp/ipykernel_37825/861545154.py:83: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selected_df['

In [9]:
# i = len(keys_list)-1
# df_name = keys_list[i]
# selected_df = dataframes[df_name]
# selected_df.reset_index(inplace=True)
# stock_df = selected_df.iloc[:,0]
# stock_df_unique = list(stock_df.unique())
# result = list(set(sorted_list) - set(stock_df_unique))
# filtered_df = total_data[total_data['instrument'].isin(result)]
# filtered_df = filtered_df.rename(columns={'volume': 'vol'})
# selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
# concat_df = pd.concat([concat_df, selected_df], ignore_index=True)

### Strategy Lake API

In [ ]:
data_api_url = ("https://dev-api.ddt-dst.cc/api/plumber/nextron/industry_prod_df")
df = pd.read_parquet(
    data_api_url,
    storage_options={
        "gcp-token": json.dumps(token),
    }
)

### Load benchmark data

In [10]:
benchmark_inst = "0050"
benchmark_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": benchmark_inst,
    },
)
benchmark_data.index.set_levels(
    pd.to_datetime(benchmark_data.index.levels[1]),
    level=1,
    inplace=True,
)
benchmark_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

benchmark_data

/tmp/ipykernel_37825/2173060872.py:11: FutureWarning: inplace is deprecated and will be removed in a future version.
  benchmark_data.index.set_levels(


open      high       low     close    volume
instrument datetime                                                    
0050       2022-06-01  119.2856  119.9775  119.0089  119.1473   6730117
           2022-06-02  118.5015  118.5015  117.7173  117.9018   4721923
           2022-06-06  118.3170  119.0089  117.5789  118.7321   4292361
           2022-06-07  118.0402  118.0402  117.0254  117.5328   5160242
           2022-06-08  118.5476  119.1011  118.2247  118.8244   4110313
...                         ...       ...       ...       ...       ...
           2024-06-24  185.5518  185.5518  182.6168  183.0645  14671365
           2024-06-25  181.0250  182.9651  179.8808  182.9651  13774562
           2024-06-26  184.0097  185.1539  183.3133  184.4077   9269483
           2024-06-27  182.6666  184.3579  182.4676  184.3579   6508242
           2024-06-28  184.3579  186.4473  183.7112  185.5021   4455951

[506 rows x 5 columns]

### Initialize Tidal
1. Initialize Tidal object
2. Add Quote data (pd.DataFrame)
3. Set strategy object (td.BaseStrategy)
4. Add metric objects (td.BaseMetic)

In [11]:
# Initialize Tidal object
tidal = td.Tidal(init_cash=1000000, slip_ticks=1, stock_config=td.StockConfig.TW, load_configs=True, reqMem="1000Mi")

# Add quote data
tidal.add_quote(quote_data)

# Set strategy object
tidal.set_strategy(td.strategy.TopkDropout(5, 2, 'new'))

# Set metric objects
tidal.add_metric(td.metric.AccountInfo())
tidal.add_metric(td.metric.AdditionalInfo())
tidal.add_metric(td.metric.PositionInfo())
tidal.add_metric(td.metric.Portfolio(benchmark_data.loc[benchmark_inst]))

### Stock Config

In [12]:
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.0004275, Min Commission:20.0, Transaction Tax:0.003}

### Instrument Configs

In [13]:
tidal.exchange.inst_configs

{'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0}}

### Config Modification

In [14]:
# Set commission to 77% off
tidal.exchange.set_stock_config(commission=0.001425 * 0.23)
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.00032775, Min Commission:20.0, Transaction Tax:0.003}

### Add New Instrument Config

In [15]:
# Add MGC config
tidal.exchange.set_config(instrument='MGC', margin=787., tick_size=0.1, tick_value=1., trade_unit=1, commission=2., min_commission=0., transaction_tax=0.)
tidal.exchange.inst_configs

{'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0},
 'MGC': InstConfig {Margin:787.0, Tick Size:0.1, Tick Value:1.0, Trade Unit:1, Commission:2.0, Min Commission:0.0, Transaction Tax:0.0}}

### Start Backtesting

In [16]:
tidal.backtest()

Tidal Backtesting: 100%|██████████| 306/306 [00:11<00:00, 26.72it/s, cash=6.11e+4, pnl=7.42e+4, position_cost=1.11e+6, value=1.25e+6] 


### Metric - AccountInfo

In [17]:
account_info = tidal.metrics["AccountInfo"].report
print(account_info)

,cash,trade_cost,position_cost,slip_cost,pnl,value,max_drawdown,turnover
datetime,,,,,,,,
2022-06-01,1000000.000000,0.000000,0.000000e+00,0.0000,0.0000,1.000000e+06,0.000000,0.0000
2022-06-02,207724.690446,259.583154,7.920157e+05,546.7264,-1271.9264,9.984685e+05,-0.001532,792015.7264
2022-06-06,207724.690446,259.583154,7.920157e+05,546.7264,4798.2736,1.004539e+06,0.000000,0.0000
2022-06-07,43247.024851,978.403149,9.592155e+05,4231.9720,-5786.5264,9.966760e+05,-0.007827,527675.1544
2022-06-08,86392.173714,2228.026286,9.118291e+05,-7444.6000,-5804.1296,9.924172e+05,-0.012067,647260.8280
...,...,...,...,...,...,...,...,...
2023-08-25,107722.527354,151737.036846,1.075663e+06,128901.6358,79105.3798,1.262491e+06,-0.057916,765267.9146
2023-08-28,107722.527354,151737.036846,1.075663e+06,128901.6358,39398.1798,1.222783e+06,-0.087546,0.0000
2023-08-29,25732.203191,152381.675209,1.152052e+06,127836.1216,63876.9066,1.241661e+06,-0.073460,419453.9142


### Metric - PositionInfo

In [18]:
position_df = tidal.metrics["PositionInfo"].report
print(position_df)

quantity      price  commission   slip_cost        pnl  \
instrument datetime                                                             
1531       2022-06-02   14000.0  13.112773   60.167960    366.4248   876.7752   
           2022-06-06   14000.0  13.112773   60.167960    366.4248  2118.5752   
           2022-06-07   14000.0  13.112773   60.167960    366.4248  -986.6248   
           2022-06-09   13000.0  13.157262   56.059804    918.6060  -918.6060   
           2022-06-16   15000.0  13.175400   64.773560   1332.0000 -2662.5000   
...                         ...        ...         ...         ...        ...   
8433       2022-08-26    3000.0  56.868710   55.916160    340.5312  8999.9688   
           2022-08-29    3000.0  56.868710   55.916160    340.5312   -73.5312   
           2022-09-01    3000.0  58.562291   57.581372  -3118.8276  1517.4276   
8928       2022-06-08    4000.0  40.246332   52.762941 -12696.2720 -4089.3280   
9911       2023-04-26    3000.0  57.691252   56.724924    345.4566  1008.1434   

                       now_price  
instrument datetime               
1531       2022-06-02    13.1754  
           2022-06-06    13.2641  
           2022-06-07    13.0423  
           2022-06-09    13.0866  
           2022-06-16    12.9979  
...                          ...  
8433       2022-08-26    59.8687  
           2022-08-29    56.8442  
           2022-09-01    59.0681  
8928       2022-06-08    39.2240  
9911       2023-04-26    58.0273  

[1522 rows x 6 columns]

In [19]:
pi_report = tidal.metrics['PositionInfo'].report
pi_report.iloc[pi_report.index.get_level_values('datetime') == '2022-06-14']

,,quantity,price,commission,slip_cost,pnl,now_price
instrument,datetime,,,,,,
1580,2022-06-14,2000.0,82.367005,53.991572,328.8104,3179.7896,83.9569
2640,2022-06-14,2000.0,68.441810,44.863607,-599.1792,-1145.6208,67.8690
5284,2022-06-14,6000.0,38.583000,75.873469,0.0000,0.0000,38.5830
6245,2022-06-14,3000.0,52.172336,51.298450,5275.6092,13270.3908,56.5958
8433,2022-06-14,5000.0,47.410600,77.694121,424.5000,-9771.0000,45.4564


### Metric - AdditionalInfo

In [20]:
tidal.metrics['AdditionalInfo'].report

new
instrument datetime            
1101       2022-06-01  0.000150
           2022-06-02  0.000140
           2022-06-06  0.000427
           2022-06-07  0.000442
           2022-06-08  0.001099
...                         ...
9958       2023-08-25  0.000335
           2023-08-28  0.000436
           2023-08-29  0.000419
           2023-08-30  0.000224
           2023-08-31  0.000419

[161141 rows x 1 columns]

### Strategy Lake Submit

In [ ]:
group = dict(zip(df["coid"].astype(str), df["tejind4_c"]))
submit_lake_backtest_result = tidal.submit_lake_backtest(account_info=account_info, position_df=position_df, benchmark_info=benchmark_data, group=group,lake_env="dev",strategy_id=19)
print(submit_lake_backtest_result)

### Trade Report

In [21]:
tidal.trade_report

,win_num,lose_num,trade_num,pos_num,win_rate,profit,loss,trade_cost,pnl
instrument,,,,,,,,,
5284,5.0,1.0,6.0,0.0,0.833333,160250.909341,-714.858054,4756.822113,159536.051287
6139,1.0,0.0,1.0,1.0,1.000000,119316.706400,0.000000,234314.493600,119316.706400
3617,1.0,2.0,3.0,0.0,0.333333,87123.360357,-9508.616791,2346.435233,77614.743567
4129,4.0,3.0,7.0,0.0,0.571429,65144.138188,-18473.537479,7329.841091,46670.600709
6470,2.0,2.0,4.0,1.0,0.500000,89329.967135,-48921.046304,175472.163968,40408.920832
2640,4.0,3.0,7.0,0.0,0.571429,29858.394197,-5998.685116,3551.151119,23859.709081
6664,1.0,0.0,1.0,0.0,1.000000,22807.648061,0.000000,780.971539,22807.648061
4938,1.0,0.0,1.0,0.0,1.000000,12861.912596,0.000000,677.836804,12861.912596
5903,1.0,0.0,1.0,0.0,1.000000,11148.739915,0.000000,692.430285,11148.739915


### Traded instruments

In [22]:
tidal.account.trades.keys()

dict_keys(['2640', '1580', '1531', '3669', '3617', '4129', '3551', '8433', '8928', '6245', '1817', '5284', '5230', '4549', '4971', '4175', '8279', '5222', '4506', '6803', '6561', '5903', '5469', '4938', '3556', '3338', '8027', '4541', '3580', '8091', '2404', '6664', '6470', '2233', '6147', '8049', '6143', '3416', '6591', '6523', '9911', '2548', '3594', '6139', '3038', '4557', '3029', '6570', '8420'])

### Trading History

In [23]:
# tidal.account.trades['3015']

### Plot chart by using Plotly

In [24]:
# tidal.analyzer.inst_chart(instrument='3015', metric_name='AdditionalInfo', plot_type=td.PlotType.LINE, scale=1.0)

### Tidal Dashboard

In [25]:
tidal.tdboard()

 * Serving Flask app 'tidal.tdboard'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:41785
 * Running on http://10.136.15.129:41785
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:10.0.10.82 - - [23/Sep/2024 16:58:44] "GET / HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [23/Sep/2024 16:58:45] "GET /static/js/main.d754b0a3.js HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [23/Sep/2024 16:58:45] "GET /static/css/main.bf4d504b.css HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [23/Sep/2024 16:58:45] "GET /images/Tidal_Logo_white.png HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [23/Sep/2024 16:58:46] "GET /api/quote/inst_list HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [23/Sep/2024 16:58:46] "GET /api/trade/trade_report HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [23/Sep/2024 16:58:46] "GET /api/metric/metric_list HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [23/Sep/2024 16:58

In [ ]:
tidal.account.position_history